In [ ]:
!mkdir sparse_vector

In [ ]:
%%capture
#@title Installs
!pip install einops
!pip install torchvision
!pip install transformers==4.26.1
!pip install genomic-benchmarks
!pip install OmegaConf

In [ ]:
%%capture
!FLASH_ATTENTION_SKIP_CUDA_BUILD=TRUE pip install flash-attn --no-build-isolation
! pip install torch --index-url https://download.pytorch.org/whl/cu118
! pip install transformers[torch] datasets

!pip install bertviz

In [ ]:
import torch
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
from einops import rearrange
from typing import Optional
from functools import partial
from torch import Tensor
from torchvision.ops import StochasticDepth
from collections import namedtuple

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch.utils import data
import random
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from collections import Counter
import pandas as pd
import numpy as np
import scipy
from tqdm import trange
from tqdm import tqdm
from datetime import datetime
import sys
import os
import seaborn as sns
import einops
from matplotlib import pyplot as plt
from joblib import Parallel, delayed, dump, load
from matplotlib import pyplot as plt
from sparse_vector.sparse_vector import SparseVector
from scipy.signal import convolve2d, convolve
import time
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score, matthews_corrcoef
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")

In [ ]:
ASSEMBLY_d = {}
chroms_d = {}
G4DNA_d = {}
DNA_d = {}

# G4

In [ ]:
ASSEMBLY = "endoquad_high_levels_small"
chroms = [f'chr{i}' for i in list(range(1, 23)) + ['X']]
G4DNA = load(f'{ASSEMBLY}.pkl')



In [ ]:
%%capture
def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'/content/drive/My Drive/DeepZ_data_creation/data/hg19_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"/content/drive/My Drive/DeepZ_data_creation/data/hg19_dna/{file}") for file in files])


DNA = {chrom:chrom_reader(chrom) for chrom in tqdm(chroms)}

In [ ]:
mode = 'hg19'
ASSEMBLY_d[mode] = ASSEMBLY
chroms_d[mode] = chroms
G4DNA_d[mode] = G4DNA
DNA_d[mode] = DNA

# Data part

In [ ]:
mode = 'hg19'
ASSEMBLY = ASSEMBLY_d[mode]
chroms = chroms_d[mode]
G4DNA = G4DNA_d[mode]
DNA = DNA_d[mode]

In [ ]:
G4DNA

{'chr1': sparse_array([0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0], dtype=<class 'numpy.int64'>),
 'chr2': sparse_array([0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0], dtype=<class 'numpy.int64'>),
 'chr3': sparse_array([0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0], dtype=<class 'numpy.int64'>),
 'chr4': sparse_array([0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0], dtype=<class 'numpy.int64'>),
 'chr5': sparse_array([0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0], dtype=<class 'numpy.int64'>),
 'chr6': sparse_array([0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0], dtype=<class 'numpy.int64'>),
 'chr7': sparse_array([0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0], dtype=<class 'numpy.int64'>),
 'chr8': sparse_array([0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0], dtype=<class 'numpy.int64'>),
 'chr9': sparse_array([0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0], dtype=<class 'numpy.int64'>),
 'chr10': sparse_array([0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0], dtype=<class 'numpy.int64'>),
 'chr11': sparse_array([0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0], dtype=<class 'nu

In [ ]:
%%capture
width = 512

np.random.seed(10)

ints_in = []
ints_out = []


for chrm in chroms:
    for st in trange(0, G4DNA[chrm].shape - width, width):
        interval = [st, min(st + width, G4DNA[chrm].shape)]
        N_count = sum([bp == "N" for bp in DNA[chrm][interval[0]:interval[1]]])
        if N_count > width / 2:
            continue
        else:
            if G4DNA[chrm][interval[0]: interval[1]].any():
                ints_in.append([chrm, int(interval[0]), int(interval[1]), 1])
            else:
                ints_out.append([chrm, int(interval[0]), int(interval[1]), 0])




print(len(ints_in))
print(len(ints_out))

ints_in_full = ints_in
ints_out_full = ints_out


In [ ]:
ints_in = ints_in_full
ints_out = [ints_out_full[i] for i in np.random.choice(range(len(ints_out_full)),
                                                    size=len(ints_in) * 20, replace=False)]
# ints_out = ints_out_full

print(len(ints_in))
print(len(ints_out))
#484 for len 1000
#9680 for len 1000

#629 for len 512
#12580 for len 5121

234
4680


In [ ]:
equalized = ints_in + ints_out

In [ ]:
divisions = list(StratifiedKFold(5, shuffle=True,
                                 random_state=42).split(equalized, [f"{elem[3]}_{elem[0]}"
                                         for i, elem
                                         in enumerate(equalized)]))

In [ ]:
dump([equalized, divisions], 'zdna_divisions.pkl', 3)

['zdna_divisions.pkl']

In [ ]:
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers

    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.

    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, chroms,
                 dna_source,
                 labels_source, intervals, tokenizer):

        self.chroms = chroms
        self.dna_source = dna_source
        self.labels_source = labels_source
        self.intervals = intervals
        self.le = LabelBinarizer().fit(np.array([["A"], ["C"], ["T"], ["G"]]))
        self.configs = {
                        'ZHUNT_AS': {
                                'CG': 0, 'GC': 1, 'CA': 0, 'AC': 1,
                                'TG': 0, 'GT': 1, 'TA': 1, 'AT': 1,
                                'CC': 0, 'GG': 0, 'CT': 1, 'TC': 1,
                                'GA': 1, 'AG': 1, 'AA': 1, 'TT': 1},
                       }
        seqs = (["A", "C", "T", "G"] +
                ['AC', 'AT', 'AG', 'CT', 'CG', 'GT'] +
                ['AAC', 'ACC', 'AAT', 'ATT', 'AAG', 'AGG',
                 'CCA', 'CAA', 'CCT', 'CTT', 'CCG', 'CGG',
                 'TTA', 'TAA', 'TTC', 'TCC', 'TTG', 'TGG',
                 'GGA', 'GAA', 'GGC', 'GCC', 'GGT', 'GTT'] +
                ['AAAC', 'AAAT', 'AAAG', 'CCCA', 'CCCT', 'CCCG',
                 'TTTA', 'TTTC', 'TTTG', 'GGGA', 'GGGC', 'GGGT'])
        self.tars = np.array([self.le.transform(list(i * 11)[:11]) for i in seqs])[:, ::-1, ::-1]
        # purine-pyrimidine
        self.tars = np.concatenate((self.tars, np.array([self.tars[4] + self.tars[9]])))
        self.tokenizer = tokenizer


    def __len__(self):
        return len(self.intervals)

    def __getitem__(self, index):
        interval = self.intervals[index]
        chrom = interval[0]
        begin = int(interval[1])
        end = int(interval[2])
        ll = list(self.dna_source[chrom][begin:end].upper())
        y = self.labels_source[interval[0]][interval[1]: interval[2]]


#         DNA PART

        dna_OHE = self.le.transform(ll)[None]

        res = pd.DataFrame(convolve(dna_OHE, self.tars)[:, 5:-5, 3].T / 11)
        res = (res.rolling(5, min_periods=1).max().values == 1).astype(int)


        X = dna_OHE.astype(np.float32)

        #K-mer part

        k_mers = seq2kmer(self.dna_source[chrom][begin:end+5].upper(), 6)
        encoded_k_mers = self.tokenizer.encode_plus(k_mers, add_special_tokens=False, max_length=512)["input_ids"]

        return torch.Tensor(X), torch.Tensor(y).long(), ll, torch.LongTensor(encoded_k_mers), (chrom, begin, end)


In [ ]:
!pip install einops transformers==4.27 peft omegaconf torch evaluate accelerate numpy scikit-learn Pillow textaugment scipy
!pip uninstall triton

In [ ]:
import torch
from transformers import BertModel, BertConfig, PreTrainedTokenizer, BasicTokenizer, BertForTokenClassification
import collections
from torch.utils.data import DataLoader
import sklearn
from sklearn.metrics import accuracy_score
from torch.nn import CrossEntropyLoss

In [ ]:
import torch
from transformers import BertModel, BertConfig, PreTrainedTokenizer, BasicTokenizer, BertForTokenClassification
import collections
from torch.utils.data import DataLoader
import sklearn
from sklearn.metrics import accuracy_score
from torch.nn import CrossEntropyLoss

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, AutoModelForTokenClassification, AutoConfig, AutoModelForSequenceClassification, AutoModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

In [ ]:
for MODEL_NUMBER in range(5):

    train_inds, test_inds = divisions[MODEL_NUMBER]
    train_intervals, test_intervals = [equalized[i] for i in train_inds], [equalized[i] for i in test_inds]

    random.shuffle(train_intervals)
    random.shuffle(test_intervals)

    train_dataset = Dataset(chroms,
                       DNA, G4DNA, train_intervals,
                        AutoTokenizer.from_pretrained(model_name, trust_remote_code=True))

    test_dataset = Dataset(chroms,
                       DNA,  G4DNA, test_intervals,
                          AutoTokenizer.from_pretrained(model_name, trust_remote_code=True))

    dump((train_dataset, test_dataset), f'zdna_seq_hg_fold{MODEL_NUMBER}.pkl')

In [ ]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):
        features, labels, sequences, input_ids, intervals = batch
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        #print(model.device, input_ids.device, labels.device)
        outputs = model(input_ids = input_ids, labels = labels)
        #print(outputs)
        loss, tr_logits = outputs['loss'], outputs['logits']
        #print(model(input_ids=ids, attention_mask=mask, labels=labels))
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 1000==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 1000 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=0.1
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels, eval_scores = [], [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            features, labels, sequences, input_ids, intervals = batch
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            outputs = model(input_ids = input_ids, labels = labels)
            loss, eval_logits = outputs['loss'], outputs['logits']
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            flattened_scores = active_logits[:,1] - active_logits[:,0]

            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            eval_scores.extend(flattened_scores)

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [id.item() for id in eval_labels]
    predictions = [id.item() for id in eval_preds]
    scores = [id.item() for id in eval_scores]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, scores

In [ ]:
device = "cuda"
lr = 1e-5
EPOCHS = 2

dir_to_pretrained_model = model_name
# config = BertConfig.from_pretrained(dir_to_pretrained_model)
# tokenizer = AutoTokenizer.from_pretrained(dir_to_pretrained_model)

for MODEL_NUMBER in range(5):
    train_dataset, test_dataset = load(f'zdna_seq_hg_fold{MODEL_NUMBER}.pkl')
    training_loader = DataLoader(train_dataset, batch_size=8, num_workers = 2)
    testing_loader = DataLoader(test_dataset, batch_size=4, num_workers = 2)


    # model = BertForTokenClassification.from_pretrained(dir_to_pretrained_model, config=config)
    model.to(device)

    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, steps_per_epoch=len(training_loader), epochs=EPOCHS)

    for epoch in range(EPOCHS):
        print(f"Training fold {MODEL_NUMBER} epoch: {epoch + 1}")
        train(epoch)
        labels, predictions, scores = valid(model, testing_loader)
        print(f'Fold {MODEL_NUMBER} validation ROC-AUC: ', roc_auc_score(labels, scores))

    print(matthews_corrcoef(labels, predictions))
    model.save_pretrained(f'zdna_bert_hg_fold_{MODEL_NUMBER}')

In [ ]:
attentions, preds, targets, seqs, bps = [], [], [], [], []

for FOLD in range(5):
    train_dataset, test_dataset = load(f'zdna_seq_hg_fold{FOLD}.pkl')
    # model.to(device)

    for example in tqdm(test_dataset):
        features, target, seq, input_ids, interval = example
        if target.numpy().sum()>0:
            with torch.no_grad():
                outputs = model(input_ids.unsqueeze(0).to(device))

                # pred = torch.softmax(outputs[-2], axis = -1)[0,:,1]
                attention = outputs[-1]


            attentions.append(attention)
            # preds.append(pred)
            targets.append(target)
            seqs.append(seq)

100%|██████████| 5934/5934 [01:26<00:00, 68.99it/s]


In [ ]:
attentions

[tensor([[[ 0.2421, -0.7800],
          [-0.3689, -0.2729],
          [-0.0716, -0.6878],
          ...,
          [ 0.2630, -0.7848],
          [ 0.4470, -0.9265],
          [ 0.4983, -0.8977]]], device='cuda:0'),
 tensor([[[ 0.1020, -0.8879],
          [ 0.5697, -0.5463],
          [ 0.1074, -0.8943],
          ...,
          [ 0.0706, -0.9089],
          [ 0.1892, -0.9749],
          [ 0.0725, -1.0392]]], device='cuda:0'),
 tensor([[[-0.8623,  0.8397],
          [-1.0498,  0.8199],
          [-0.8107,  0.8459],
          ...,
          [-0.9206,  0.7595],
          [-0.9837,  0.8191],
          [-0.7887,  0.7983]]], device='cuda:0'),
 tensor([[[ 0.6607, -0.8751],
          [ 0.6501, -0.6878],
          [ 0.5870, -0.8618],
          ...,
          [ 0.7157, -1.3706],
          [ 0.7409, -1.1427],
          [ 0.8408, -1.0418]]], device='cuda:0'),
 tensor([[[ 1.8510, -2.2776],
          [ 2.4449, -2.6689],
          [ 2.3043, -2.4963],
          ...,
          [ 1.4249, -1.8050],
     

In [ ]:
!pip install bertviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 14.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [ ]:
from collections import defaultdict
kmer2pred = defaultdict(int)
kmer2att = defaultdict(float)


for attention, target, seq in tqdm(zip(attentions, targets, seqs)):
    kmer = seq2kmer(''.join(seq), 6).split(' ')
    #print(kmer)
    att = attention[-1][0,:,:,:]

    for idx in range(512-5):
        if target[idx]>0:
            kmer2pred[kmer[idx]]+=1

            for head in range(12):
                c_att = att[head,idx,:].numpy()
                for att_idx in range(512-5):
                    kmer2att[kmer[att_idx]]+=c_att[att_idx]

0it [00:00, ?it/s]


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [ ]:
sorted_pred = [t[0] for t in sorted(kmer2pred.items(), key=lambda item: -item[1])]
kmer2att2 = {k:int(kmer2att[k]) for k in kmer2att}
sorted_att  = [t[0] for t in sorted(kmer2att2.items(), key=lambda item: -item[1])]

In [ ]:
for idx, kmer in enumerate(sorted_att):
    print(idx+1, kmer, sorted_pred.index(kmer)+1)
    if idx>100:
        break

In [ ]:
chroms = [f'chr{i}' for i in list(range(1, 23)) + ['X', 'Y']]
ZDNA = load(f'ZDNA_2016.pkl')
black_list = load(f'blacklist_hg19.pkl')

def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'../data/hg19_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"../data/hg19_dna/{file}") for file in files])

DNA = {chrom:chrom_reader(chrom) for chrom in tqdm(chroms)}

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1054/1054 [00:00<00:00, 77419.29it/s]


In [ ]:
device = "cuda"
torch.cuda.empty_cache()
models = []

# config = AutoConfig.from_pretrained(dir_to_pretrained_model)

for MODEL_NUMBER in range(5):
    dir_to_pretrained_model = f'dnabert_hg_fold_{MODEL_NUMBER}'
    model = BertForTokenClassification.from_pretrained(dir_to_pretrained_model).to(device)
    model.to(device)
    model.eval()
    models.append(model)

In [ ]:
width = 512
k_mer_pad = 5

def final_prediction(chrom):

    intervals = []
    ends = []

    prediction = [np.zeros(ZDNA[chrom].shape, dtype=np.float32) for i in range(5)]


    for st in range(0, ZDNA[chrom].shape - width, width):
        interval = [st, min(st + width + k_mer_pad, ZDNA[chrom].shape)]
        intervals.append([chrom, interval[0], interval[1]])

    ends.append([chrom, ZDNA[chrom].shape // width * width, ZDNA[chrom].shape])


    full_dataset = Dataset(chroms, [i for i in feature_names if i.upper() in intersect],
                           DNA, DNA_features,  ZDNA, intervals, tokenizer)

    ends_dataset = Dataset(chroms, [i for i in feature_names if i.upper() in intersect],
                           DNA, DNA_features, ZDNA, ends, tokenizer)

    params = {'batch_size':32, 'num_workers':5, 'shuffle':False}
    load_predict = data.DataLoader(full_dataset, **params)

    params = {'batch_size':1, 'num_workers':1, 'shuffle':False}
    load_ends = data.DataLoader(ends_dataset, **params)

    with torch.no_grad():

        for features, labels, sequences, input_ids, intervals in tqdm(load_predict):
            input_ids = input_ids.to(device)
            outputs = [torch.softmax(model(input_ids = input_ids)['logits'],axis = -1).cpu().numpy()[:,:,1] for model in models]

            for ind, interval in enumerate(zip(intervals[0],
                                         intervals[1].cpu().numpy(),
                                         intervals[2].cpu().numpy())):
                #print(ind, interval)
                #print(len(prediction), len(outputs), len(models))
                for i in range(len(models)):
                    prediction[i][interval[1]:interval[2]-k_mer_pad] = outputs[i][ind, :]

    for model_i in range(len(models)):
        dump(prediction[model_i], f'../models/predictions/new_mod_hg_{model_i}_{chrom}', 3)


In [ ]:
for chrom in chroms[19:]:
    print(f"BEGIN CHROM {chrom}")
    final_prediction(chrom)

BEGIN CHROM chr20


  0%|                                                                                                                         | 0/3847 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `trun

In [ ]:
chroms = [f'chr{i}' for i in list(range(8, 23)) + ['X', 'Y']]

In [ ]:
ZDNA = load(f'../data/hg19_zdna/sparse/ZDNA_2016.pkl')
black_list = load(f'../data/hg19_zdna/sparse/blacklist_hg19.pkl')

def chrom_reader(chrom):
    files = sorted([i for i in os.listdir(f'../data/hg19_dna/') if f"{chrom}_" in i])
    return ''.join([load(f"../data/hg19_dna/{file}") for file in files])


DNA = {chrom:chrom_reader(chrom) for chrom in tqdm(chroms)}

In [ ]:
equalized, divisions = load('hg_divisions.pkl')

In [ ]:
com_len = sum([len(DNA[chrom]) for chrom in chroms])
sums = []

for chrom in tqdm(chroms):
    loc_sum = []
    for model_num in range(5):
        vec = load(f"../models/predictions/new_mod_hg_{model_num}_{chrom}")
        loc_sum.append(vec.sum())
    sums.append(loc_sum)

multipliers = np.array(sums).sum(axis=0) / com_len

In [ ]:
for chrom in tqdm(chroms):
    vecs = np.array([load(f"../models/predictions/new_mod_hg_{i}_{chrom}")
                     for i in range(5)])
    res_vec = (vecs / multipliers[:, None]) * multipliers.mean()
    mean_vec = res_vec.mean(axis=0)

    test_ints = []
    for MODEL_NUMBER in range(5):
        train_inds, test_inds = divisions[MODEL_NUMBER]
        train_intervals, test_intervals = [equalized[i] for i in train_inds], [equalized[i] for i in test_inds]
        test_ints.extend([(MODEL_NUMBER, inter) for inter in test_intervals if inter[0] == chrom])

    for model_num, inters in test_ints:
        mean_vec[inters[1]: inters[2]] = res_vec[model_num, inters[1]: inters[2]]
    dump(mean_vec, f"../models/predictions/new_mod_hg_res_{chrom}", 3)

In [ ]:
for chrom in tqdm(chroms):
    vec = load(f"../models/predictions/new_mod_hg_res_{chrom}")
    iids = np.where(black_list[chrom].data == 1)[0]
    for i, j in zip(black_list[chrom].indices[iids], black_list[chrom].indices[iids + 1]):
        vec[i:j] = 0
    dump(vec, f"../models/predictions/new_mod_hg_res_{chrom}", 3)